In [51]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
import random

In [52]:
from sympy.polys.domains.compositedomain import CompositeDomain
from sympy.polys.domains.domainelement import DomainElement
from sympy.polys.domains.simpledomain import SimpleDomain
from sympy.polys.domains.ring import Ring
from sympy.polys.domains.field import Field
from sympy.polys.rings import PolyRing
from sympy.polys.orderings import lex
from sympy.polys.polyutils import PicklableWithSlots
from sympy.polys.polyerrors import CoercionFailed, DomainError
from sympy.polys.domains.characteristiczero import CharacteristicZero
from sympy.polys.domains.modularinteger import ModularIntegerFactory
from sympy.polys.domains.fractionfield import FractionField

class EllipticCurveGroupElement(DomainElement):
    def __init__(self, x, y, group):
        self.x = x
        self.y = y
        self.group = group

    def __str__(self):
        return r'EC({}, {})'.format(self.x, self.y)
    
    def _latex(self, printer):
        return r'\left({},{},{}\right)'.format(printer._print(self.x), printer._print(self.y), printer._print(self.group))

    def __add__(self, other):
        if self.group != other.group:
            raise TypeError('groups are not equal')

        if self.x == other.x and self.y == other.y:
            if self.y == 0:
                return self.group.new(0, 0)
            else:
                s = (3 * self.x**2 + self.group.a) / (2 * self.y)
                x = s**2 - 2 * self.x
                y = s * (self.x - x) - self.y
                return self.group.new(x, y)
        else:
            if self.x == other.x:
                return self.group.new(0, 0)
            else:
                s = (other.y - self.y) / (other.x - self.x)
                x = s**2 - self.x - other.x
                y = s * (self.x - x) - self.y
                return self.group.new(x, y)

    def __mul__(self, other):
        if not isinstance(other, int):
            raise TypeError('other is not an integer')

        if other < 0:
            return -self * (-other)

        if other == 0:
            return self.group.new(0, 0)

        if other == 1:
            return self

        if other % 2 == 0:
            return (self + self) * (other // 2)
        else:
            return self + self * (other - 1)

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return self.group.new(self.x, -self.y)

    def __sub__(self, other):
        return self + (-other)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __ne__(self, other):
        return self.x != other.x or self.y != other.y

    def __hash__(self):
        return hash((self.x, self.y))

    def __getitem__(self, index):
        if index == 0:
            return self.x
        elif index == 1:
            return self.y
        else:
            raise IndexError('index out of range')
    
    def parent(self):
        return self.group

class EllipticCurveGroup(CompositeDomain):
    rep = 'EC'
    alias = 'EC'

    dom = None
    a, b = None, None

    # y^2 = x^3 + ax + b
    def __init__(self, a, b, dom):
        if dom.is_Field:
            self.dom = dom
        elif dom.has_assoc_Field:
            self.dom = dom.field
        else:
            raise TypeError('dom is not a field')

        self.a = a
        self.b = b

    def __str__(self):
        return r'EC(y^2 = x^3 + {}x + {} over {})'.format(self.a, self.b, self.dom)
    
    def _latex(self, printer):
        return r'\mathbb{EC}_{%s}[y^2 = x^3%s%s]' % (
            printer._print(self.dom),
            r'+ %sx' % printer._print(self.a) if self.a else r' + x' if self.a == self.dom.one else '',
            r' + %s' % printer._print(self.b) if self.b else '')

    def verify(self, x, y):
        return y**2 == x**3 + self.a * x + self.b

    def new(self, x, y):
        if not self.verify(x, y):
            raise ValueError('point is not on the curve')
        return EllipticCurveGroupElement(x, y, self)

    __call__ = new

    @property
    def zero(self):
        return self.new(0, 0)

    def random(self):
        while True:
            x = self.dom.random()
            y = self.dom.sqrt(x**3 + self.a * x + self.b)
            if y is not None:
                return self.new(x, y)

class ZRing(Ring, SimpleDomain):
    rep = 'ZZ'
    alias = 'ZZ'

    is_IntegerRing = is_ZZ = True
    is_Numerical = True
    is_PID = True

    has_assoc_Ring = True
    has_assoc_Field = True

    dom = None
    mod = None

    def __init__(self, mod, symmetric=True):
        self.dom = ZZ
        self.dtype = ModularIntegerFactory(mod, self.dom, symmetric, self)
        self.zero = self.dtype(0)
        self.one = self.dtype(1)
        self.mod = mod
        self.is_Ring = True
        self.is_FiniteField = self.is_FF = self.is_Field = ntheory.isprime(mod)
        self.has_assoc_Ring = not self.is_Field
    
    def __str__(self):
        return 'Z%s' % self.mod

    def _latex(self, printer):
        if self.is_Field:
            return r'\mathbb{F}_{%s}' % self.mod
        return r'\mathbb{Z}_{%s}' % self.mod

    def __hash__(self):
        return hash((self.__class__.__name__, self.mod))
    
    def __eq__(self, other):
        return isinstance(other, ZRing) and self.mod == other.mod
    
    def __ne__(self, other):
        return not self == other
    
    def characteristic(self):
        return self.mod
    
    def get_ring(self):
        if self.is_Field:
            raise DomainError("there's no ring associated with %s" % self)
    
    def get_field(self):
        if self.is_Field:
            return self
        else:
            return self.frac_field(self)
    
    def poly_ring(self, *symbols, order=lex):
        from sympy.polys.domains.polynomialring import PolynomialRing
        return PolynomialRing(self, symbols, order)
    
    def frac_field(self, *symbols, order=lex):
        from sympy.polys.domains.fractionfield import FractionField
        return FractionField(self, symbols, order)
    
    def algebraic_field(self, *extension, alias=None):
        return self.get_field().algebraic_field(*extension, alias=alias)

    def __getitem__(self, symbols):
        if hasattr(symbols, '__iter__'):
            return self.poly_ring(*symbols)
        else:
            return self.poly_ring(symbols)
    
    def __call__(self, *args):
        if len(args) == 1 and type(args[0]) == int:
            return self.dtype(args[0])
        else:
            return self.frac_field(*args)
    
    def to_sympy(self, a):
        return Integer(int(a))

    def from_sympy(self, a):
        if a.is_Integer:
            return self.dtype(self.dom.dtype(int(a)))
        elif a.is_Float and int(a) == a:
            return self.dtype(self.dom.dtype(int(a)))
        else:
            raise CoercionFailed("expected an integer, got %s" % a)

    def from_FF(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ(a.val, K0.dom))

    def from_FF_python(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ_python(a.val, K0.dom))

    def from_ZZ(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ_python(a, K0))

    def from_ZZ_python(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ_python(a, K0))

    def from_QQ(K1, a, K0=None):
        if a.denominator == 1:
            return K1.from_ZZ_python(a.numerator)

    def from_QQ_python(K1, a, K0=None):
        if a.denominator == 1:
            return K1.from_ZZ_python(a.numerator)

    def from_FF_gmpy(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ_gmpy(a.val, K0.dom))

    def from_ZZ_gmpy(K1, a, K0=None):
        return K1.dtype(K1.dom.from_ZZ_gmpy(a, K0))

    def from_QQ_gmpy(K1, a, K0=None):
        if a.denominator == 1:
            return K1.from_ZZ_gmpy(a.numerator)

    def from_RealField(K1, a, K0):
        p, q = K0.to_rational(a)
        if q == 1:
            return K1.dtype(K1.dom.dtype(p))

    def sqrt(self, a):
        rt = ntheory.sqrt_mod(int(a), self.mod)
        if rt is None:
            return None
        else:
            return self.dtype(rt)
    
    def is_square(self, a):
        return ntheory.is_quad_residue(int(a), self.mod)
    
    def random(self):
        return self.dtype(random.randint(0, self.mod - 1))
    
    def log(self, a, b):
        return self.dtype(ntheory.discrete_log(int(a), int(b), self.mod))

    def gcdex(self, a, b):
        g, x, y = ntheory.gcdex(int(a), int(b))
        return self.dtype(g), self.dtype(x), self.dtype(y)
    
    def gcd(self, a, b):
        return self.dtype(gcd(int(a), int(b)))
    
    def lcm(self, a, b):
        return self.dtype(lcm(int(a), int(b)))
    
    def factorial(self, a):
        return self.dtype(factorial(int(a)))

class ModularPolynomialRing(PicklableWithSlots, DomainElement):
    mod, dom, _parent = None, None, None

    __slots__ = ('val',)

    def parent(self):
        return self._parent

    def __init__(self, val):
        if isinstance(val, self.__class__):
            self.val = val.val % self.mod
        else:
            self.val = self.dom(val) % self.mod

    def __hash__(self):
        return hash((self.val, self.mod))

    def __repr__(self):
        return "%s(%s)" % (self.__class__.__name__, self.val)

    def __str__(self):
        return "(%s) mod (%s)" % (self.val, self.mod)

    def _latex(self, printer):
        return r'(%s)\text{ mod }(%s)' % (printer._print(self.val), printer._print(self.mod))

    def __pos__(self):
        return self

    def __neg__(self):
        return self.__class__(-self.val)

    @classmethod
    def _get_val(cls, other):
        if isinstance(other, cls):
            return other.val
        else:
            try:
                return cls.dom.convert(other)
            except CoercionFailed:
                return None

    def __add__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(self.val + val)
        else:
            return NotImplemented

    def __radd__(self, other):
        return self.__add__(other)

    def __sub__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(self.val - val)
        else:
            return NotImplemented

    def __rsub__(self, other):
        return (-self).__add__(other)

    def __mul__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(self.val * val)
        else:
            return NotImplemented

    def __rmul__(self, other):
        return self.__mul__(other)

    def __truediv__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(self.val * self._invert(val))
        else:
            return NotImplemented

    def __rtruediv__(self, other):
        return self.invert().__mul__(other)
    
    def __floordiv__(self, other):
        return self.__truediv__(other)
    
    def __rfloordiv__(self, other):
        return self.__rtruediv__(other)

    def __mod__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(self.val % val)
        else:
            return NotImplemented

    def __rmod__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.__class__(val % self.val)
        else:
            return NotImplemented

    def __pow__(self, exp):
        if not exp:
            return self._parent.one

        if exp < 0:
            val, exp = self.invert(), -exp
        else:
            val = self

        return self.pow(val, int(exp))
    
    def pow(self, base, exp):
        res = self._parent.one
        while exp > 0:
            if exp & 1:
                res = (res * base) % self.mod
            exp >>= 1
            base = (base * base) % self.mod
        return res
    
    def __eq__(self, other):
        val = self._get_val(other)

        if val is not None:
            return self.val == val
        else:
            return False
    
    def __ne__(self, other):
        return not self.__eq__(other)

    def __lt__(self, other):
        if type(other) == int:
            return False
        raise NotImplemented

    def __bool__(self):
        return bool(self.val)

    def invert(self):
        return self.pow(self.val, self._parent.order() - 1)

    def __invert__(self):
        return self.invert()

_modular_polynomial_ring_cache = {}

def ModularPolynomialRingFactory(_mod, _dom, parent):
    try:
        _mod = _dom.convert(_mod)
    except CoercionFailed:
        raise ValueError("expected an polynomial, got %s" % _mod)

    class cls(ModularPolynomialRing):
        mod, dom = _mod, _dom
        _parent = parent

    return cls

class GaloisField(Field):
    rep = 'GF'
    alias = 'GF'

    is_PolynomialRing = is_Poly = True

    has_assoc_Ring = False
    has_assoc_Field = True

    dom = None
    mod = None

    def __init__(self, mod, domain, symbol=None, order=None):
        if not domain.is_Field:
            raise TypeError('domain is not a field')
        
        ring = domain.poly_ring(symbol, order=order)

        self.ring = ring
        self.dtype = ModularPolynomialRingFactory(mod, ring, self)

        self.mod = ring.from_sympy(mod)
        self.gens = ring.gens
        self.ngens = ring.ngens
        self.symbols = ring.symbols
        self.domain = ring.domain

        self.one = self.dtype(ring.one)
        self.zero = self.dtype(ring.zero)
    
    def __str__(self):
        return 'GF(%s)' % self.mod
    
    def _latex(self, printer):
        return r'\mathbb{GF}_{%s}(%s^%s)' % (printer._print(self.mod), self.domain.characteristic(), self.mod.degree())
    
    def __hash__(self):
        return hash((self.__class__.__name__, self.mod))
    
    def __eq__(self, other):
        return isinstance(other, GaloisField) and self.mod == other.mod
    
    def __ne__(self, other):
        return not self == other
    
    def new(self, element):
        return self.dtype(element)
    
    def __call__(self, element):
        return self.dtype(element)
    
    def characteristic(self):
        return self.domain.characteristic()
    
    def order(self):
        return self.domain.characteristic() ** self.mod.degree() - 1
    
    def get_ring(self):
        raise DomainError("there's no ring associated with %s" % self)
    
    def get_field(self):
        return self

    def poly_ring(self, *symbols, order=lex):
        from sympy.polys.domains.polynomialring import PolynomialRing
        return PolynomialRing(self, symbols, order)
    
    def frac_field(self, *symbols, order=lex):
        from sympy.polys.domains.fractionfield import FractionField
        return FractionField(self, symbols, order)
    
    def algebraic_field(self, *extension, alias=None):
        return self.get_field().algebraic_field(*extension, alias=alias)
    
    def __getitem__(self, symbols):
        if hasattr(symbols, '__iter__'):
            return self.poly_ring(*symbols)
        else:
            return self.poly_ring(symbols)
    
    def __call__(self, *args):
        if len(args) == 1 and type(args[0]) == self.dtype:
            return self.dtype(self.domain.dtype(int(args[0])))
        else:
            return self.frac_field(*args)
    
    def to_sympy(self, a):
        return self.ring.to_sympy(a.val)
    
    def from_sympy(self, a):
        return self.dtype(self.ring.from_sympy(a))

    def random(self):
        deg = self.mod.degree()
        coe = [((i,), self.domain.random()) for i in range(deg)]
        return self.new(self.ring.new(coe))
    
    def legendre(self, a):
        return a ** (self.order() // 2)
    
    def cpow(self, x, w2, exp):
        xr, xi = x, self.one
        rr, ri = self.one, self.zero
        while exp > 0:
            if exp & 1:
                rr, ri = rr * xr + ri * xi * w2, rr * xi + ri * xr
            exp >>= 1
            xr, xi = xr * xr + xi * xi * w2, 2 * xr * xi
        assert ri == self.zero
        return rr
    
    # cipolla's algorithm
    def sqrt(self, a):
        if a == self.zero:
            return self.zero
        if self.legendre(a) == -self.one:
            return None
        while True:
            b = self.random()
            w2 = b * b - a
            if self.legendre(w2) == -self.one:
                return self.cpow(b, w2, self.order() // 2 + 1)

    def is_square(self, a):
        return self.legendre(a) != -self.one

class ComplexDomain(Domain):
    pass

class MatrixRing(Domain):
    pass

class TensorGroup(Domain):
    # 可以[x,y],[z]写指标
    # * => einsum
    # 可以使用微分算子
    # 可以实现LaTeX输出
    pass

class FunctionGroup(Domain):
    # 半群，可以用于函数的复合
    # 输入输出类型相同则含幺，可数乘
    pass

class GeometryAlgebra(Domain):
    # 可以表示向量、点、线、面、体积
    # 可以表示旋转、平移、缩放
    # 可以表示曲线、曲面
    rep = 'GA'
    alias = 'GA'

    class Element(DomainElement):
        dim, dom = None, None
        val = None

        def __init__(self, val):
            self.val = val
        
        def __hash__(self):
            return hash((self.__class__.__name__, self.val))
        
        @classmethod
        def hat(cls, v):
            return r'\hat{%s}' % v
        
        def __str__(self):
            for k, v in self.val.items():
                return '%s%s' % (v, k)

    def __init__(self, dim, dom):
        self.dim = dim
        self.dom = dom
    
    def __str__(self):
        return 'GA(%s) over %s' % (self.dim, self.dom)
    
    def __hash__(self):
        return hash((self.__class__.__name__, self.dim, self.dom))
    
    def __eq__(self, other):
        return isinstance(other, GeometryAlgebra) and self.dim == other.dim and self.dom == other.dom
    
    def __ne__(self, other):
        return not self == other
    
    def new(self, num, tags):
        code = 0
        for tag in tags:
            if tag > self.dim:
                raise ValueError('tag out of dimension')
            code |= 1 << tag
        return self.Element({code: num})
    
    def __call__(self, num, tags):
        return self.new(num, tags)

class LieGroup(Domain):
    # 可以表示旋转、平移、缩放
    # 可以表示曲线、曲面
    pass

class MultiplicationTableSemigroup(Domain):
    pass

In [53]:
x = Symbol('x')
F = GaloisField(x**2 + 1, ZRing(11), symbol=x)

EC = EllipticCurveGroup(5, 7, F)
display(Math(latex(EC.random())))

display(Math(latex(EC)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

- F_p, GF(p^n) [p != 2] => Cipolla
- 域 + 两两可比较(仅要求可传递) => Index Calculus
- Matrix(交换环) => Cayley-Hamilton Theorem
- 整环 => Miller-Rabin, Pollard-Rho Factorization
- 域 + 循环乘法子群 => FFT/NTT
- 域 + (x^2 + 1)扩域后存在循环乘法子群 => FHT
- 乘法半群 + 有限阶 => Baby-Step Giant-Step
- 加法阿贝尔群 + 有限阶 => Pollard-Rho Discrete Logarithm